In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
include("9msctrl.jl")

┌ Info: Recompiling stale cache file C:\Users\Kaijie\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


η2int (generic function with 1 method)

In [29]:
Re = 500
u0 = rand(9)
dadt = zeros(9)
q    = zeros(9)
dqdt = zeros(9)
T     = 5000
timestep = 0.5
ts = 0:timestep:T
store = RAMStorage(zeros(9))
B = [0,0,0,0,1,0,0,0,0]
α = 0.1



function minus_energy_grad(t, a, dadt, v, dvdt)
    dvdt[1] += -2*(a[1] - 1)
    for i = 2:9
        dvdt[i] += -2*a[i]
    end
    return dvdt
end


objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
function objfun(t, u, dudt, I, dIdt)
    return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
end
function objfun(xq::Coupled)
    return objfun(xq[1])
end


function Objective_fun_CTRL(η::AbstractVector)
    f = NineModeSystemCTRL(Re, ts, η)
    ϕ = flow(couple(f,objfun),RK4(couple(zeros(9),zeros(1))), TimeStepConstant(timestep))
    I0 = Float64[0.0]
    u0_local = copy(u0)
    ϕ(couple(u0_local, I0), extrema(ts))
    return I0[1]/T + α.*η2int(η, timestep)
end

function Objective_fun_CTRL1(η::AbstractVector)
    
#     function firstel(u::AbstractVector)
#         return u[1]
#     end
#     function firstel(t, u, dudt, I, dIdt)
#         return dIdt[1] = firstel(u)
#     end
#     function firstel(xq::Coupled)
#         return firstel(xq[1])
#     end
    
    f = NineModeSystemCTRL(Re, ts, η)
    F = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestep))
    I0 = Float64[0.0]
    u0_local = copy(u0)
    mon = Monitor(couple(zeros(9), zeros(1)), objfun)
    out = F(couple(u0_local,I0), extrema(ts), reset!(mon))
    times(mon)
    samples(mon)
    plot(times(mon), samples(mon))
    xlabel(L"t")
    ylabel(L"objfun(t)")
    return out
end

function Objective_fun_Gradient!(G::AbstractVector,η::AbstractVector)
    f = NineModeSystemCTRL(Re, ts, η)
    ϕ = flow(f,RK4(zeros(9)), TimeStepConstant(timestep))
    u0_local = copy(u0)
    ϕ(u0_local, (0, T), reset!(store))
    
    h = NineModeSystemLin(Re, true, minus_energy_grad)
    ψ_adj = flow(h, RK4(zeros(9), ContinuousMode(true)), TimeStepFromStorage(timestep))
    mon = Monitor(zeros(9), q -> -B[5]*q[5])
    ψ_adj(zeros(9), store, (T, 0), reset!(mon))
    G .= samples(mon)[end:-1:1] .+ 2.0.*α.*η
end


result = optimize(Objective_fun_CTRL, Objective_fun_Gradient!, zeros(length(ts)),GradientDescent(),Optim.Options(f_tol = 1e-6,
                             store_trace = true,
                             show_trace = true))

# η = zeros(length(ts))
# f = NineModeSystemCTRL(Re, ts, η)
# ϕ = flow(f,RK4(zeros(9)), TimeStepConstant(timestep))
# u0_local = copy(u0)
# ϕ(u0_local, (0, T), reset!(store))
    
# h = NineModeSystemLin(Re, true, minus_energy_grad)
# ψ_adj = flow(h, RK4(zeros(9), ContinuousMode(true)), TimeStepFromStorage(timestep))
# mon = Monitor(zeros(9), q -> -B[5]*q[5])
# ψ_adj(zeros(9), store, (T, 0), reset!(mon))
# G = samples(mon)
# times(mon)
# samples(mon)


# h(0.0, a, dadt, q, dqdt)
# f = NineModeSystem(Re)
# ϕ = flow(f, RK4(zeros(9)), TimeStepConstant(1))
# ϕ(a, (0, T), reset!(store))
# ψ_adj = flow(h, RK4(zeros(9), ContinuousMode(true)), TimeStepFromStorage(1))
# mon = Monitor(zeros(9), q -> q[5])
# ψ_adj(q, store, (T, 0), reset!(mon))
# samples(mon)
# times(mon)

# u0 = rand(9)
# f  = NineModeSystem(1000)

# objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)

# function objfun(t, u, dudt, I, dIdt)
#     return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
# end

# function objfun(xq::Coupled)
#     return objfun(xq[1])
# end

# ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(1))
# I0 = Float64[0.0]
# ϕ(couple(u0, I0), (0, 10000))
# #I0/10000
# mon = Monitor(couple(zeros(9), zeros(1)), objfun)

# I1 = Float64[0]
# @time ϕ(couple(u0, I1), (0, 10000), reset!(mon))

# times(mon)
# samples(mon)

# figure(1)
# clf()

# plot(times(mon), samples(mon))
# xlabel(L"t")
# ylabel(L"cost a_1(t)")

Iter     Function value   Gradient norm 
     0     3.027942e-01     1.630085e+17
 * time: 0.0009999275207519531
     1     3.027942e-01     1.630085e+17
 * time: 6.519000053405762
     2     3.027942e-01     1.630085e+17
 * time: 6.5350000858306885


 * Status: success

 * Candidate solution
    Minimizer: [-8.97e-19, -2.19e-18, -3.19e-18,  ...]
    Minimum:   3.027942e-01

 * Found with
    Algorithm:     Gradient Descent
    Initial Point: [0.00e+00, 0.00e+00, 0.00e+00,  ...]

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 1.0e-06
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 1.63e+17 ≰ 1.0e-08

 * Work counters
    Seconds run:   7  (vs limit Inf)
    Iterations:    2
    f(x) calls:    116
    ∇f(x) calls:   116


In [30]:
η = result.minimizer
Objective_fun_CTRL1(η)
# η = sin.(ts)
# f = NineModeSystemCTRL(Re, ts, η)
# ϕ = flow(f,RK4(zeros(9)), TimeStepConstant(timestep))
# u0_local = copy(u0)
# ϕ(u0_local, (0, T), reset!(store))
    
# h = NineModeSystemLin(Re, true, minus_energy_grad)
# ψ_adj = flow(h, RK4(zeros(9), ContinuousMode(true)), TimeStepFromStorage(timestep))
# mon = Monitor(zeros(9), q -> -B[5]*q[5])
# ψ_adj(zeros(9), store, (T, 0), mon)
# η = samples(mon)

2-element Coupled{2,Tuple{Array{Float64,1},Array{Float64,1}}}:
 [0.9999998739672529, 9.693381511766581e-10, -1.5718265398619432e-12, 7.140393197383667e-10, -4.901111317972972e-10, -1.6927937931659703e-11, -5.329953079122461e-12, -1.3074987385774196e-12, -5.142519138067939e-20]
 [1513.9711562164966]                                                                                                                                                                                                 

In [33]:
a = zeros(1)
a = vcat(a,[1,2,3])

4-element Array{Float64,1}:
 0.0
 1.0
 2.0
 3.0